<a href="https://colab.research.google.com/github/asifahsaan/NLP-Natural-Language-Processing-/blob/main/Info_Retrieval_VSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>
<center>

|Title:     | Information Retrieval Assignment 2 |
|         :-            |                             :-:                               |
| **Resource Person:**  | Dr. Iqra Safder                                      |
| **Course Title:**     | Information Retrieval and Text Mining                                                 |
| **Credit Hours:**     | 3(3,0)                                                        |
| **Work Objective:**     | Implementation of VSM on large corpus                                                  |
<br>
            |          **Submitted By** |

|Sr. # | Name                   | Registration No. | Email Address           |
| :-   | :-                     | :-               | :-                      
| 1    | Asif Ahsan | 20L-2370         | l202370@lhr.nu.edu.pk |
<br>
<br>

# **Vector Space Model (TF-IDF Weightage Model)**

<h3 style = 'color:purple;'>Vector Space Model (TF-IDF Weightage Model)</h3>

$$ f(q,d) = sim(q,d) =  \sum_{i=1}^n x_iy_i $$ 
q = (x_1,.....,x_n) <br>
d = (y_1,.....,y_n) <br>
x_i = count of word W_i in query. <br>
y_i = TF-IDF of word W_i in doc i.e $$ y_i = C(W_i,doc) * log_2 \frac {M+1} {k} $$
M = number of documents in the collection <br>
k = document frequency for every word in corpus

### Mounting drive to access data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing important libraries

In [ ]:
import os
import re
import nltk
import string
import math
import operator
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
from operator import itemgetter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### ***Unzipping the corpusDatausDataus data***

In [ ]:
!unzip /content/drive/MyDrive/IR_Dataset/ACL_Docs.zip

Streaming output truncated to the last 5000 lines.
  inflating: ACL_Dcos/W11-4106.pdf.txt  
  inflating: ACL_Dcos/W11-4107.pdf.txt  
  inflating: ACL_Dcos/W11-4108.pdf.txt  
  inflating: ACL_Dcos/W11-4109.pdf.txt  
  inflating: ACL_Dcos/W11-4110.pdf.txt  
  inflating: ACL_Dcos/W11-4111.pdf.txt  
  inflating: ACL_Dcos/W11-4112.pdf.txt  
  inflating: ACL_Dcos/W11-4113.pdf.txt  
  inflating: ACL_Dcos/W11-4114.pdf.txt  
  inflating: ACL_Dcos/W11-4115.pdf.txt  
  inflating: ACL_Dcos/W11-4200.pdf.txt  
  inflating: ACL_Dcos/W11-4201.pdf.txt  
  inflating: ACL_Dcos/W11-4202.pdf.txt  
  inflating: ACL_Dcos/W11-4203.pdf.txt  
  inflating: ACL_Dcos/W11-4204.pdf.txt  
  inflating: ACL_Dcos/W11-4205.pdf.txt  
  inflating: ACL_Dcos/W11-4206.pdf.txt  
  inflating: ACL_Dcos/W11-4207.pdf.txt  
  inflating: ACL_Dcos/W11-4208.pdf.txt  
  inflating: ACL_Dcos/W11-4300.pdf.txt  
  inflating: ACL_Dcos/W11-4301.pdf.txt  
  inflating: ACL_Dcos/W11-4302.pdf.txt  
  inflating: ACL_Dcos/W11-4303.pdf.txt  
  infl

### ***Showing data retrieved***

In [ ]:
import os
path = "/content/ACL_Dcos"
os.chdir(path)

listFiles = os.listdir("/content/ACL_Dcos")
print(listFiles)

Documents = []
dicList = []
TF_IDF = []

['W12-1704.pdf.txt', 'W12-6328.pdf.txt', 'W15-1609.pdf.txt', 'W15-4006.pdf.txt', 'W15-2614.pdf.txt', 'W15-0120.pdf.txt', 'W12-6103.pdf.txt', 'W13-3110.pdf.txt', 'W14-0202.pdf.txt', 'W13-3816.pdf.txt', 'W15-3820.pdf.txt', 'W99-0104.pdf.txt', 'W12-1912.pdf.txt', 'W15-1105.pdf.txt', 'W15-1213.pdf.txt', 'W14-1901.pdf.txt', 'W13-2128.pdf.txt', 'W94-0304.pdf.txt', 'W98-0705.pdf.txt', 'W14-1007.pdf.txt', 'W12-4107.pdf.txt', 'W13-4603.pdf.txt', 'W11-3901.pdf.txt', 'W15-0103.pdf.txt', 'W15-4506.pdf.txt', 'W12-2906.pdf.txt', 'W15-3056.pdf.txt', 'W13-3700.pdf.txt', 'W13-2248.pdf.txt', 'W13-2512.pdf.txt', 'W15-3711.pdf.txt', 'W95-0103.pdf.txt', 'W13-3609.pdf.txt', 'W12-2106.pdf.txt', 'W14-3348.pdf.txt', 'W13-4812.pdf.txt', 'W12-0501.pdf.txt', 'W13-2203.pdf.txt', 'W15-2302.pdf.txt', 'W15-0201.pdf.txt', 'W14-1814.pdf.txt', 'W94-0103.pdf.txt', 'W12-2025.pdf.txt', 'W94-0112.pdf.txt', 'W15-4110.pdf.txt', 'W99-0909.pdf.txt', 'W97-1507.pdf.txt', 'W93-0214.pdf.txt', 'W14-2908.pdf.txt', 'W13-4824.pdf.txt',

### **1. Reading Data in wordList Function**




In [ ]:
def wordList(doc):
  f =  open(doc, "r", encoding ="unicode_escape")
  try:
    dataRead=f.read()
    word_list = dataRead.split()
    return word_list

  except:
    return []

### **2. Preprocessing Corpus Data**

In [ ]:
def removePuncs(wordList):
  word_list = []

  stopWords = stopwords.words('english')

  for word in wordList:
    newWord = word.translate(str.maketrans('', '', string.punctuation))
    newWord = newWord.lower()

    if newWord not in stopWords:
        word_list.append(newWord)

  return word_list

### **3. Term Frequency of Documents**

In [ ]:
def termFrequencyInDoc(wordList):
    
  termFrequency = {}
  
  for Word in wordList:
    
    if Word not in termFrequency:
      termFrequency[Word] = 0
    
    termFrequency[Word] += 1

  return termFrequency

In [ ]:
def corpus(dicList):
 corpusData = [] # a list that will store words of the vocabulary

 for doc in dicList: #iterate through documents 
    for word in doc: #go through each word in the current doc
        if not word in corpusData: 
            corpusData.append(word)
            
#  print('\t\tTotal words in corpus' , len(corpusData))
 return corpusData

### **4. Word Document Frequency**

In [ ]:
def wordDocFre(dicList):

  corpusData = corpus(dicList)
  docFreq = {} 
  for word in corpusData:
    for doc in dicList:
        if word in doc.keys():
          if word in docFreq.keys():
            docFreq[word] += 1 
          else:
            docFreq[word] = 1

  return docFreq


### **Q5. Inverse Document Frequency Function**

In [ ]:
def inverseDocFre(dicList,base):

  M = len(Documents) #number of documents in the collection
  IDF = {} #inverse_document frequency for every word in corpus
  for word in dicList:
       IDF[word] = math.log((M+1/dicList[word]),base)
 
  return IDF


### **6. TF_IDF Function**

In [ ]:
def tfidf():

  for file in os.listdir():     # iterate through all file
    if file.endswith(".txt"):
      file_path = f"{path}/{file}"

      word_list = wordList(file_path)
      if len(word_list) > 0:
        Documents.append(file)
       
        PreprocessData = removePuncs(word_list)
        
        TF = termFrequencyInDoc(PreprocessData)
        dicList.append(TF)

  DF = wordDocFre(dicList)
  IDF = inverseDocFre(DF, 10)

  for doc in dicList:   # Calculate TF_IDF
    tfidf_dict = {}
    for word in doc:
      tfidf_dict[word] = IDF[word] * doc[word]
    TF_IDF.append(tfidf_dict)

In [ ]:
tfidf()

### **7. VSM (Vector Space Model)**

In [ ]:
def vectorSpaceModel(query, count):
  query = query.split()
  
  print("\nQUERY :", count)
  Q = []
  for word in query:
    Q.append(word.lower())

  print('   • System Query: ', query)
  TF_Query = {}     # Term Frequency of query
  
  for word in Q:
    if word not in TF_Query:
      TF_Query[word] = 0

    TF_Query[word] += 1
  
  print('   • Term Frequency of Query: ',TF_Query)

  Relevance = {}
  
  index = 0

  for doc in TF_IDF:
    Relevance[Documents[index]] = 0

    for word in TF_Query:
      if word in doc:
        Relevance[Documents[index]] += TF_Query[word] * doc[word]

    index += 1

  TopRelevantScoreDocs = dict(sorted(Relevance.items(), key=operator.itemgetter(1),reverse=True)[:5])  #Reterive the names of 5 top raked docs

  Keys = list(TopRelevantScoreDocs.keys())

  if len(Keys) > 5:
    TempList = []
    for i in range(5):
      TempList.append(Keys[i])
    
    Keys = TempList

  if len(Keys) > 0:
      print('\n---> Top 5 with highest relevant score documents are:')
      print(TopRelevantScoreDocs)
      print('\t\t\t\t\t\t____________________________________________________\n')
  else:
    print('No Document Found!')
    print('\t\t\t\t\t\t____________________________________________________\n')

**Running Queries using VSM**

In [ ]:
systemQueries = ['LDA', 'Topic modelling', 'Generative models', 'Semantic relationships between terms', 'Natural Language Processing', 'Text Mining', 'Translation model', 'Learning procedures for the lexicon', 'Semantic evaluations', 'System results and combination']
count = 0

for Query in systemQueries:
  count = count+1
  vectorSpaceModel(Query, count)


QUERY : 1
   • System Query:  ['LDA']
   • Term Frequency of Query:  {'lda': 1}

---> Top 5 with highest relevant score documents are:
{'W13-1914.pdf.txt': 140.52453272314165, 'W13-0908.pdf.txt': 133.12850468508154, 'W13-5412.pdf.txt': 129.4304906660515, 'W13-5416.pdf.txt': 99.84637851381116, 'W14-3419.pdf.txt': 92.45035047575108}
						____________________________________________________


QUERY : 2
   • System Query:  ['Topic', 'modelling']
   • Term Frequency of Query:  {'topic': 1, 'modelling': 1}

---> Top 5 with highest relevant score documents are:
{'W14-3110.pdf.txt': 465.94970923688066, 'W14-3358.pdf.txt': 436.3656011597708, 'W14-4410.pdf.txt': 421.5735464524158, 'W14-3202.pdf.txt': 391.98943837530595, 'W12-6006.pdf.txt': 373.4993706376189}
						____________________________________________________


QUERY : 3
   • System Query:  ['Generative', 'models']
   • Term Frequency of Query:  {'generative': 1, 'models': 1}

---> Top 5 with highest relevant score documents are:
{'W97-